In [1]:
import pandas as pd
import ast

**Dataset**

In [2]:
df = pd.read_csv("homework_10_exercise_4_results.csv")
df

,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
0,0,0.9105,0.7419,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657223675
1,1,0.9579,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657223675
2,2,0.8947,0.7742,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657223675
3,3,0.9632,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,[],1657223675
4,4,0.8526,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.010,Adam,0.3,3,random,arm,[],1657223675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,5,0.6421,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723
76,6,0.9526,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723
77,7,0.8895,0.9355,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723
78,8,0.8421,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


**The results, for each set of hyperparameters when run in the several environments.**

The results on any given platform are consistent when the GPU is not enabled.  When the GPU is enabled, results tend to vary.

In [9]:
for index in df.panel_index.unique():
    display(df[df.panel_index == index])

,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
0,0,0.9105,0.7419,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657223675
10,0,0.9263,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,[],1657225324
20,0,0.8842,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227028
30,0,0.8789,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227191
40,0,0.9263,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227352
50,0,0.9158,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
60,0,0.9263,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
70,0,0.8947,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
1,1,0.9579,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657223675
11,1,0.9632,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,[],1657225324
21,1,0.9684,0.7742,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227028
31,1,0.9421,0.8387,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227191
41,1,0.9684,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227352
51,1,0.9684,0.8065,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
61,1,0.9579,0.8387,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
71,1,0.9579,0.8387,"(3, 3)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
2,2,0.8947,0.7742,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,[],1657223675
12,2,0.9053,0.8710,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,[],1657225324
22,2,0.9000,0.7742,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227028
32,2,0.8947,0.7742,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227191
42,2,0.8842,0.8065,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227352
52,2,0.9053,0.8710,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
62,2,0.9158,0.9032,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
72,2,0.9000,0.9032,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
3,3,0.9632,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,[],1657223675
13,3,0.9053,0.8387,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,x86_64,[],1657225324
23,3,0.9000,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227028
33,3,0.9000,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227191
43,3,0.8526,0.8387,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227352
53,3,0.9000,0.9032,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
63,3,0.8105,0.8065,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
73,3,0.8737,0.8387,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
4,4,0.8526,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,arm,[],1657223675
14,4,0.8421,0.9032,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,x86_64,[],1657225324
24,4,0.8789,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227028
34,4,0.8684,0.7419,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227191
44,4,0.8211,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227352
54,4,0.7579,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
64,4,0.7263,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
74,4,0.8474,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.01,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
5,5,0.6684,0.6129,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,[],1657223675
15,5,0.6421,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,x86_64,[],1657225324
25,5,0.7053,0.7097,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227028
35,5,0.7105,0.7097,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227191
45,5,0.7263,0.7097,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227352
55,5,0.6421,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
65,5,0.6421,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
75,5,0.6421,0.4839,"(5, 5)","(1, 1)","(2, 2)",0.001,SGD,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
6,6,0.9211,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,[],1657223675
16,6,0.9053,0.9355,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,[],1657225324
26,6,0.9211,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227028
36,6,0.9421,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227191
46,6,0.9421,0.7419,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227352
56,6,0.9526,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
66,6,0.9526,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
76,6,0.9526,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
7,7,0.9263,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,[],1657223675
17,7,0.9368,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,[],1657225324
27,7,0.8421,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227028
37,7,0.8632,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227191
47,7,0.8579,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227352
57,7,0.8579,0.9355,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
67,7,0.8789,0.9677,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
77,7,0.8895,0.9355,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
8,8,0.9053,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,[],1657223675
18,8,0.8526,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,x86_64,[],1657225324
28,8,0.8895,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227028
38,8,0.9105,0.7097,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227191
48,8,0.8842,0.7419,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227352
58,8,0.8316,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
68,8,0.8632,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
78,8,0.8421,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,yes,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
9,9,0.9579,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,[],1657223675
19,9,0.8947,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,x86_64,[],1657225324
29,9,0.9263,0.7419,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227028
39,9,0.8421,0.7419,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227191
49,9,0.9632,0.7742,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,arm,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657227352
59,9,0.8737,0.8387,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
69,9,0.9000,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
79,9,0.8895,0.8065,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,3,no,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723


**All results, sort by validation accuracy**

In [58]:
df.groupby('panel_index', as_index=False)['validation_accuracy'].mean().sort_values('validation_accuracy', ascending=False)

,panel_index,validation_accuracy
7,7,0.846788
2,2,0.834688
3,3,0.830662
6,6,0.818550
1,1,0.814538
4,4,0.806450
8,8,0.798400
9,9,0.782262
0,0,0.745988
5,5,0.584700


In [6]:
df.sort_values('validation_accuracy', ascending=False)[0:20]

,panel_index,train_accuracy,validation_accuracy,kernel_size,strides,pool_size,learning_rate,optimizer,brightness,contrast_factor,flip_on_train,processor,gpu,batch
67,7,0.8789,0.9677,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
16,6,0.9053,0.9355,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.1,3,random,x86_64,[],1657225324
57,7,0.8579,0.9355,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
77,7,0.8895,0.9355,"(5, 5)","(1, 1)","(2, 2)",0.001,Adam,0.3,2,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723
72,2,0.9000,0.9032,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229723
14,4,0.8421,0.9032,"(5, 5)","(1, 1)","(2, 2)",0.010,Adam,0.3,3,random,x86_64,[],1657225324
62,2,0.9158,0.9032,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229451
53,3,0.9000,0.9032,"(5, 5)","(1, 1)","(3, 3)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
52,2,0.9053,0.8710,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,"[LogicalDevice(name='/device:GPU:0', device_ty...",1657229105
12,2,0.9053,0.8710,"(5, 5)","(2, 2)","(2, 2)",0.001,Adam,0.3,3,random,x86_64,[],1657225324
